# XBRL US API - Python Example  
This notebook contains example Python code to use the XBRL US Application Programming Interface (API) (https://xbrl.us/home/use/xbrl-api/)    
  
**Made by:** [Ties de Kok](https://www.tiesdekok.com), [Beth Blankespoor](https://foster.uw.edu/faculty-research/directory/elizabeth-blankespoor/), and [Roman Chychyla](https://people.miami.edu/profile/rxc303@miami.edu)

## API documentation

The documentation on XBRL US API is available here:

https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails

## Imports

First, import (load) supporting python libraries

In [ ]:
import os, re, sys # to work with operating system and text 
import json # to read a popular data representation format, JSON
import requests # to handle HTTP (web) requests
import pandas as pd # for tabular manipulation and computation
import numpy as np # for numerical computations
import getpass # to (interactively) request password input for a user 

## Generating an access token

To access XBRL US API, we need to use an access token (for user authentication purposes). Access tokens can be requested at XBRL US website (after registration).

For this session we will create a temporary access token (for demo purposes only). Input your email address when asked. This access token will expire after 60 minutes.

In [ ]:
CREDENTIAL_TYPE = 'TEMP'

Alternatively, you can obtain your own credentials here: https://xbrl.us/home/use/xbrl-api-community/#provisioning

After that, if you use this script on your own computer, we recommend using the JSON file as described in option (a) below. If you are using Binder, we recommend using option (b).

>**Option a:**
>1. Update 'login_cred.json' with your `client_id`, `client_secret`, `username`, and `password`;
>2. Set `CREDENTIAL_TYPE` to `LOCAL`;
>3. Input your password when asked.

>**Option b:**
>1. Set `CREDENTIAL_TYPE` to `CLOUD`;
>2. Input your details when asked,



In [ ]:
# the following code generates an access token for XBRL US API

if CREDENTIAL_TYPE == 'TEMP':
    user_email = input(prompt="Please type your email address here: ")
    access_token = requests.get('https://tdekok-xbrlapi.builtwithdark.com/gettoken?platform=aaa-{}'.format(user_email)).text.replace('"', "")
elif CREDENTIAL_TYPE in ['LOCAL', 'CLOUD']:
    endpoint = 'https://api.xbrl.us'
    endpoint_auth = endpoint + '/oauth2/token'
    
    if CREDENTIAL_TYPE == 'LOCAL':
        with open('login_cred.json', 'r') as f:
            login_cred = json.loads(f.read())
            client_id = login_cred['client_id']
            client_secret = login_cred['client_secret']
            username = login_cred['username']
            password = login_cred['password']
            
    else:
        client_id = input(prompt='Please input your client id here:')
        client_secret = getpass.getpass(prompt = 'Please input your client secret here:')
        username = input(prompt='Please input your username here:')
        
#     password = getpass.getpass(prompt = 'Password: ')
    
    body_auth = {'grant_type' : 'password', 
                'client_id' : client_id, 
                'client_secret' : client_secret, 
                'username' : username,
                'password' : password,
                'platform' : 'uw-ipynb'}
    res = requests.post(endpoint_auth, data=body_auth)
    auth_json = res.json()
    access_token = auth_json['access_token']
else:
    print('Invalid credential type! Use TEMP, LOCAL, or CLOUD. See the instructions above.')

## Making an API Query

To request XBRL data from XBRL US, we need to create an API query that specified information to be retrieved.

The Python function below generates a query to XBRL US and returns results in a tabular format. The function has the following input arguments:

- access_token - the token we generated in the previous step;
- firm_ciks - a list of company central index keys (CIKs);
- years - a list of data years (time period);
- report_types - a list of report types to consider (e.g., 10-K or 10-Q);
- get extensions - has to be set to either:
    - 'FALSE' - output XBRL elements that are not extensions;
    - 'TRUE' - output only extension XBRL. The argument xbrl_elements list will be ignored;
- xbrl_elements - a list of XBRL elements (e.g., NetIncomeLoss for net income). This list will be ignored if get_extensions is set to 'TRUE';
- with_dimensions - has to be set to either:
    - 'FALSE' - output XBRL elements *without* dimensions only;
    - 'TRUE' - output XBRL elements *with* dimensions only;
    - 'ALL' - output XBRL elements *with* and *without* dimensions;

In [ ]:
def execute_query(access_token, firm_ciks, years, report_types, get_extensions, xbrl_elements, with_dimensions):
    
    # below is fields (variable) to be output by XBRL US API. 
    # this list can be modified if other/additional information is needed
    # see XBRL US API documentation for a list of all possible fields
    fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'period.fiscal-year',
         'report.document-type',
         'report.filing-date',
         'concept.local-name',
         'fact.value',
         'dimensions.count', 
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'dts.id',
         'fact.id',
         'fact.has-dimensions'
         'period.instant',
         'concept.id',     
         'unit',
         'fact.decimals',
         'dimension.namespace',
         'member.namespace',
         ]
    
    search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'


    params = {  
         'period.fiscal-period': 'Y',
         'period.fiscal-year': ','.join(years),
         'unit': 'USD',
         'entity.cik': ','.join(firm_ciks),
         'report.document-type': ','.join(report_types)
         }  
    
    if get_extensions == 'TRUE':
        params['concept.is-base'] = 'FALSE'
    else:
        params['concept.local-name'] =  ','.join(xbrl_elements)
    
    if with_dimensions == 'ALL':
        dimension_options = ['TRUE', 'FALSE']
    else:
        dimension_options = [with_dimensions]

    all_res_list = []
    for dimensions_param in dimension_options:
        print('Getting the data for: "fact.has-dimensions" = {}'.format(dimensions_param))
        ### Every request will return a max of 2000 results. So we loop until all results are retrieved. 
        done_retrieving_all_results = False
        offset = 0
        while not done_retrieving_all_results:
            params['fact.has-dimensions'] = dimensions_param
            params['fields'] = ','.join(fields) + ',fact.offset({})'.format(offset) 
            res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})

            ## Interpret as JSON
            res_json = res.json()

            ## Get the results
            ### Retrieve the data list
            res_list = res_json['data']

            ### Add to the results
            all_res_list += res_list

            ## Pagination check
            paging_dict = res_json['paging']
            if paging_dict['count'] >= 2000:
                offset += paging_dict['count']
            else:
                done_retrieving_all_results = True

    ## convert to a DataFrame
    res_df = pd.DataFrame(all_res_list)
    ## remove duplicates; sometimes the same item is reported multiple times throughout the document
    res_df.drop_duplicates(subset = ['entity.name','period.fiscal-year', 'report.filing-date', 'concept.local-name', 'dimension.local-name', 'member.local-name', 'fact.value'], inplace = True)
    ## sort data
    res_df = res_df.sort_values(by=['entity.name','period.fiscal-year','report.filing-date','concept.local-name','dimension.local-name']).reset_index(drop = True)
    ## reorder table columns
    first_columns = ['entity.name', 'period.fiscal-year', 'report.filing-date', 'concept.local-name', 'fact.value', 'unit','dimension.local-name', 'member.local-name','dimensions.count']
    columns = first_columns + [c for c in res_df.columns if c not in first_columns]
    res_df = res_df[columns]
    print('\nNumber of results that meet the criteria: {}'.format(len(res_df)))

    return res_df

## Example: Getting Revenue from Contracts with Customers data from XBRL 10-K filings

### Define the companies you'd like

In [ ]:
firm_ciks =     [
                 '0000320193', ## Apple (AAPL)  
                 '0001018724', ## Amazon (AMZN)
                 '0001067983', ## Berkshire Hathaway(BRK)
                 '0001166691', ## Comcast (CMCSA)
                 '0001326380', ## GameStop (GME)
                 '0000354950', ## Home Depot (HD)
                 '0000200406', ## Johnson & Johnson (JNJ)
                 '0001045810', ## NVIDIA Corporation (NVDA)
                 '0001318605', ## Tesla (TSLA)
                 '0000731766', ## UnitedHealth Group (UNH)
                ]

### Define the years you'd like

In [ ]:
years = ['2019','2020'] ## Use commas between for multiple years, e.g., '2018','2019','2020'
#years = [str(2013 + i) for i in range(8)] ## Years 2013 to 2020

### Specify the report types that you want

In [ ]:
report_types = ['10-K', '10-K/A']

### Specify whether to output extension XBRL elements

In [ ]:
get_extensions = 'FALSE'

### Define the XBRL elements (tags) you'd like 

Revenue reported under the new standard (Accounting Standards Update No. 2014-09, *Revenue from Contracts with Customers* (Topic 606)) is represented by two concepts in the FASB's US GAAP Financial Reporting Taxonomy: *RevenueFromContractWithCustomerExcludingAssessedTax* and *RevenueFromContractWithCustomerIncludingAssessedTax*. We will attempt to extract Revenue data using these two concepts.

In [ ]:
xbrl_elements = [
     'RevenueFromContractWithCustomerExcludingAssessedTax',
     'RevenueFromContractWithCustomerIncludingAssessedTax'
                ]

### Define the fields you would like returned

Below, we choose fields (variable) to be output by XBRL US API. This list can be modified if other/additional information is needed. See XBRL US API [documentation](https://xbrlus.github.io/xbrl-api/#/fact/getFactDetails) for the complete list of available fields.

### Specify if you want dimensions, no dimension, or all values

In [ ]:
with_dimensions = 'FALSE'  ## TRUE for require dimensions, FALSE for no dimensions, ALL for all values

### Execute query

Finally, we can execute an API query and save results to *res_df* pandas dataframe.

In [ ]:
res_df = execute_query(access_token, firm_ciks, years, report_types, get_extensions, xbrl_elements, with_dimensions)

### Display results

Next, we will output the results. To facilitate table display, we will "hide" some columns when displaying the result and limit the output to 40 records.

In [ ]:
#choose which columns to hide
columns_to_hide = ['entity.cik', 'fact.id', 'fact.decimals', 'dimension.namespace', 'member.namespace']
columns_to_show = [column for column in res_df.columns if column not in columns_to_hide]

#display the first 40 results
res_df[columns_to_show].head(40)

## Example: Getting Total Revenue

In the above example, we were able to get Revenue data for only 7 out of 10 firms in the list. 

Consider the Income Statement from Tesla's fiscal year 2020 10-K filing:

https://www.sec.gov/ix?doc=/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm

Tesla's Total Revenues of \\$31,536m is comprised of four items represented by the following XBRL facts:

- Automative sales, \\$26,184m (*RevenueFromContractWithCustomerExcludingAssessedTax* with *tsla:AutomotiveSegmentMember* extension member in product/service axis (dimension));
- Automotive leasing, \\$1,052m (*OperatingLeasesIncomeStatementLeaseRevenue*);
- Energy generation and storage, \\$1,994m (*RevenueFromContractWithCustomerExcludingAssessedTax* with *tsla:EnergyGenerationAndStorageSegmentMember* extension member in product/service axis (dimension));
- Services and other, \\$2,306m (*tsla:SalesRevenueServicesAndOtherNet* (extension XBRL element);

That is, two components of Total Revenue, Automative leasing and Services and other, are not reported using ASC 606 revenue recognition. Because of this, the Total Revenues are reported using a more general *Revenues* XBRL concept rather than *RevenueFromContractWithCustomerExcludingAssessedTax*.

To extract total revenue data from all the filings, we simply need to include the *Revenues* concept in the list of XBRL concepts to be retrieved by XBRL US API query.

In [ ]:
# update the list of XBRL elements with Revenues concept:
xbrl_elements_total_revenues = xbrl_elements + ['Revenues']

# re-run the query with the new XBRL elements list:
res_df_total_revenues = execute_query(access_token, firm_ciks, years, report_types, get_extensions, xbrl_elements_total_revenues, with_dimensions)

#display the first 50 results
res_df_total_revenues[columns_to_show].head(50)

Note that there are multiple records for the same company-year pair. For example:
- There are two records for Berkshire Hathaway in 2020 fiscal year, *Revenues* and *RevenueFromContractWithCustomerExcludingAssessedTax*. These two facts are from the same 10-K filing. *Revenues* corresponds to the Total Revenues while *RevenueFromContractWithCustomerExcludingAssessedTax* is a subset of Total Revenues that does not include items such as Revenues from Leasing, Revenues from Insurance Premiums, etc. To reduce it to one fact, we can keep the fact with the highest dollar value in a given firm's filing as the firm's Total Revenues for this filing.
- There are two records for Amazon in 2019 with the same concept *RevenueFromContractWithCustomerExcludingAssessedTax*. Their values are identical and both reflect Total Revenues. These facts are from different filings (2019 and 2020 10-Ks). We can keep only one value of Total Revenues per firm-year by selecting the concept reported in the most recent filing (in case there was a restatement).


In [ ]:
res_df_output = (res_df_total_revenues
                 # sort concepts by filing ID and dollar valued(in the descending order) and keep only one observation per filing
                 .sort_values(['dts.id', 'period.fiscal-year','fact.value'], ascending=[True,True,False]).drop_duplicates(['dts.id', 'period.fiscal-year'])
                 # sort concepts by the report filing date (in the descending order) and keep only one observation per firm-year
                 .sort_values(['entity.cik', 'period.fiscal-year','report.filing-date'], ascending=[True,True,False]).drop_duplicates(['entity.cik', 'period.fiscal-year'])
                  # sort concepts by the entity name and fiscal year
                 .sort_values(['entity.name', 'period.fiscal-year'])
                 # reset record index (this is optional)
                 .reset_index(drop = True) )


#show the final table
res_df_output[columns_to_show].head(20)

### Send output to csv

Python's Pandas library provides simple means to save Pandas tables (data frames) to Excel, Stata, comma values separated (CSV), and other data formats.

In [ ]:
res_df_output.to_csv('XBRL_Revenues.csv')     

## If running in Binder, click on the Jupyter icon/name in the upper left corner to see your files, 
## select the file you want, and click Download.